In [2]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import Literal
from duckduckgo_search import DDGS
import uuid
import os

print(" All imports successful")

# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print(" API key loaded")

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)
print(f" LLM initialized: {llm.model_name}")

 All imports successful
 API key loaded
 LLM initialized: gpt-4o-mini


In [4]:
@tool
def weather_tool(city: str) -> str:
    """
    Get simulated weather information for a given city.
    Use this when user asks about weather in a specific location.
    
    Args:
        city: Name of the city to get weather for
        
    Returns:
        Weather information for the city
        
    Examples:
        - "Lagos" → Temperature and conditions for Lagos
        - "New York" → Temperature and conditions for New York
    """
    # Simulated weather data for demo purposes
    weather_data = {
        "lagos": " Lagos: 28°C, Partly Cloudy, Humidity: 75%",
        "new york": " New York: 5°C, Snowing, Humidity: 65%",
        "london": " London: 8°C, Rainy, Humidity: 80%",
        "tokyo": " Tokyo: 12°C, Clear, Humidity: 55%",
        "sydney": " Sydney: 25°C, Sunny, Humidity: 60%",
        "paris": " Paris: 6°C, Cloudy, Humidity: 70%",
    }
    
    city_lower = city.lower().strip()
    
    if city_lower in weather_data:
        return weather_data[city_lower]
    else:
        return f"Weather for {city}: Estimated 20°C, Moderate winds. (City not in database)"

print(" Weather tool created")

 Weather tool created


### Tool 2: Dictionary Tool

In [5]:
@tool
def dictionary_tool(word: str) -> str:
    """
    Look up the definition of a word in the dictionary.
    Use this when user asks to define a word or explain its meaning.
    
    Args:
        word: The word to look up
        
    Returns:
        Definition of the word
        
    Examples:
        - "ephemeral" → Definition of ephemeral
        - "serendipity" → Definition of serendipity
    """
    # Simple dictionary with common words
    definitions = {
        "ephemeral": "Lasting for a very short time; transitory.",
        "serendipity": "The occurrence of events by chance in a happy or beneficial way.",
        "ubiquitous": "Present, appearing, or found everywhere.",
        "melancholy": "A feeling of pensive sadness, typically with no obvious cause.",
        "pragmatic": "Dealing with things in a realistic and practical way based on actual circumstances.",
        "paradigm": "A typical example or pattern of something; a model.",
        "eloquent": "Fluent or persuasive in speaking or writing.",
        "meticulous": "Showing great attention to detail; very careful and precise.",
        "benevolent": "Kind, generous, and doing good; charitable.",
        "cacophony": "A harsh, discordant mixture of sounds.",
    }
    
    word_lower = word.lower().strip()
    
    if word_lower in definitions:
        return f" {word.capitalize()}: {definitions[word_lower]}"
    else:
        return f"Definition for '{word}': Word not found in database. Please try another word."

print(" Dictionary tool created")

 Dictionary tool created


### Tool 3: Web Search Tool (DuckDuckGo)

In [6]:
@tool
def web_search_tool(query: str) -> str:
    """
    Search the web using DuckDuckGo for information.
    Use this when user asks for current news, research, or information not in your knowledge.
    
    Args:
        query: The search query
        
    Returns:
        Search results summary
        
    Examples:
        - "latest AI news" → Returns recent AI news
        - "how to make bread" → Returns bread recipes and instructions
    """
    try:
        ddgs = DDGS()
        results = ddgs.text(query, max_results=3)
        
        if results:
            search_summary = f"🔍 Search results for '{query}':\n\n"
            for i, result in enumerate(results, 1):
                title = result.get('title', 'No title')
                body = result.get('body', 'No description')
                search_summary += f"{i}. **{title}**\n   {body[:150]}...\n\n"
            return search_summary
        else:
            return f"No search results found for '{query}'. Try a different search term."
    except Exception as e:
        return f" Search error: {str(e)}. Please try again."

print(" Web search tool created")

 Web search tool created


---
## Section 3: Bind Tools to LLM

In [7]:
# Create list of all tools
tools = [weather_tool, dictionary_tool, web_search_tool]

# Bind tools to LLM - this allows the LLM to see and use all three tools
llm_with_tools = llm.bind_tools(tools)

print(f" LLM bound to {len(tools)} tools")
print(f"   Tools available: {[tool.name for tool in tools]}")

 LLM bound to 3 tools
   Tools available: ['weather_tool', 'dictionary_tool', 'web_search_tool']


---
## Section 4: Build the Agent Graph

In [8]:
# Define system prompt
sys_msg = SystemMessage(content="""You are a helpful assistant with access to three tools:
1. Weather tool - for weather information
2. Dictionary tool - for word definitions
3. Web search tool - for current information and research

Guidelines:
- Use weather_tool when asked about weather in a city
- Use dictionary_tool when asked to define or explain words
- Use web_search_tool for current news, research, or general information
- For simple greetings or general knowledge questions, answer without tools
- Always provide clear, helpful responses

Choose the most appropriate tool for each query. Only use tools when necessary.""")

def assistant(state: MessagesState) -> dict:
    """
    Assistant node - decides whether to use tools or answer directly.
    """
    messages = [sys_msg] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

print(" Assistant node defined")

 Assistant node defined


In [9]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """
    Conditional routing - decides if we should use tools or end.
    """
    last_message = state["messages"][-1]
    
    # If LLM made tool calls, go to tools node
    if last_message.tool_calls:
        return "tools"
    
    # Otherwise, we're done
    return "__end__"

# Build the graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

# Compile with memory
memory = MemorySaver()
agent = builder.compile(checkpointer=memory)

print(" Agent graph built and compiled with memory")

 Agent graph built and compiled with memory


In [11]:
# Create a unique thread for this session
thread_id = f"multi-tool-session-{str(uuid.uuid4())[:8]}"

# print("=" * 80)
# print(" MULTI-TOOL AGENT - INTERACTIVE MODE")
# print("=" * 80)
# print(f"\nThread ID: {thread_id}")
# print("\n Examples of what you can ask:")
# print("   - 'What is the weather in Lagos?'")
# print("   - 'Define the word ephemeral'")
# print("   - 'Search for latest AI news'")
# print("   - 'What is the capital of France?' (no tool needed)")
# print("\nType 'exit' to end the conversation\n")
# print("-" * 80 + "\n")

turn = 1
while True:
    # Get user input
    user_input = input(f" You (Turn {turn}): ").strip()
    
    # Check for exit command
    if user_input.lower() == "exit":
        print("\n Conversation ended. Thank you!")
        break
    
    # Skip empty messages
    if not user_input:
        print(" Please enter a message.\n")
        continue
    
    # Invoke the agent
    try:
        result = agent.invoke(
            {"messages": [HumanMessage(content=user_input)]},
            config={"configurable": {"thread_id": thread_id}}
        )
        
        # Extract and display the final response
        for message in result["messages"]:
            if isinstance(message, AIMessage) and not message.tool_calls:
                print(f"\n Agent: {message.content}\n")
                break
            elif isinstance(message, AIMessage) and message.tool_calls:
                tool_name = message.tool_calls[0]['name']
                print(f"\n Agent: [Using tool: {tool_name}]")
        
        print("-" * 80 + "\n")
        turn += 1
        
    except Exception as e:
        print(f"\n Error: {str(e)}\n")
        print("-" * 80 + "\n")
        continue


 Agent: [Using tool: weather_tool]

 Agent: The current weather in Lagos is 28°C with partly cloudy skies and a humidity level of 75%.

--------------------------------------------------------------------------------


 Conversation ended. Thank you!


In [12]:
# View the conversation history
session_state = agent.get_state(
    config={"configurable": {"thread_id": thread_id}}
)

print("\n" + "=" * 80)
print(" CONVERSATION HISTORY & TOOL USAGE")
print("=" * 80)
print(f"\nTotal messages in session: {len(session_state.values['messages'])}\n")

for i, message in enumerate(session_state.values["messages"], 1):
    msg_type = type(message).__name__
    
    if isinstance(message, HumanMessage):
        print(f"[{i}]  Human: {message.content}")
    elif isinstance(message, AIMessage):
        if message.tool_calls:
            tool_info = message.tool_calls[0]
            print(f"[{i}]  AI (Tool Call): {tool_info['name']}({tool_info['args']})")
        else:
            content_preview = message.content[:80] + "..." if len(message.content) > 80 else message.content
            print(f"[{i}]  AI: {content_preview}")
    elif isinstance(message, ToolMessage):
        content_preview = message.content[:80] + "..." if len(message.content) > 80 else message.content
        print(f"[{i}]  Tool Result: {content_preview}")

print("\n" + "=" * 80)
print(" Session complete!")


 CONVERSATION HISTORY & TOOL USAGE

Total messages in session: 4

[1]  Human: tell me about lagos weather
[2]  AI (Tool Call): weather_tool({'city': 'Lagos'})
[3]  Tool Result:  Lagos: 28°C, Partly Cloudy, Humidity: 75%
[4]  AI: The current weather in Lagos is 28°C with partly cloudy skies and a humidity lev...

 Session complete!
